## Question Generation

This notebook contains functions for automated question generation.

In [1]:
# imports
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from dotenv import dotenv_values
env_vars = dotenv_values('../functions/.env')

# temperature controls the randomness of the generated text
# use temperature = 0.0 for development
chat = AzureChatOpenAI(
    deployment_name=env_vars["SEC_AZURE_DEPLOYMENT_ID"],
    openai_api_key=env_vars["SEC_AZURE_API_KEY"],
    openai_api_base=env_vars["SEC_AZURE_ENDPOINT"],
    openai_api_version='2023-05-15', temperature=0.0)

In [2]:
LESSON_TEXT = '''“Form follows function” is a philosophy that many industries follow. In architecture, this means that buildings should be constructed to support the activities housed within them. For example, a skyscraper should include several elevator banks. A hospital should have an easily accessible emergency room.

Our natural world also utilizes the principle of form following function, especially in cell biology. The structure of cell components and organelles is directly related to their function. For example, ribosomes, the organelles responsible for protein synthesis, are found in high numbers in eukaryotic cells that synthesize large amounts of protein, such as those in the pancreas that produce digestive enzymes. Here the form (high number of ribosomes) follows the function (protein synthesis). Apart from ribosomes, eukaryotic cells have numerous other organelles. These organelles are specialized compartments inside the cells, and similar to the organs in your body, each organelle has a unique role. In this lesson, we will dive into several examples of organelles and discuss their specific functions.


Ribosomes

Ribosomes are the cellular structures responsible for protein synthesis. They may group together into clusters (polyribosomes) or individual ribosomes may float freely in the cytoplasm. They may be attached to the plasma membrane's cytoplasmic side or the endoplasmic reticulum's cytoplasmic side and the nuclear envelope's outer membrane. They are large protein and RNA complexes, each consisting of two subunits, large and small (Figure 1). Ribosomes receive their “orders” for protein synthesis from the nucleus where the DNA transcribes into messenger RNA (mRNA). The mRNA travels to the ribosomes, which translate the code provided by the sequence of the nitrogenous bases in the mRNA into a specific order of amino acids linked together. Amino acids are the building blocks of proteins.


Mitochondria

Mitochondria (singular = mitochondrion) are bean-shaped structures each with a double membrane that are often called “powerhouses” of cells because they make adenosine triphosphate (ATP), the cell’s main energy-carrying molecule. ATP represents the cell's short-term stored energy. Cellular respiration is the process of making ATP using the chemical energy in glucose and other nutrients. In mitochondria, this process uses oxygen and produces carbon dioxide as waste. In fact, the carbon dioxide that you exhale with every breath comes from the cellular reactions that produce carbon dioxide as a byproduct.

In keeping with our theme of form following function, it is important to note that muscle cells have a very high concentration of mitochondria that produce ATP. Your muscle cells need considerable energy to keep your body moving. When your cells don’t get enough oxygen, they make small amounts of ATP and also produce lactic acid.


Peroxisomes

Peroxisomes are small, round organelles enclosed by single membranes. They carry out oxidation reactions that break down fatty acids and amino acids. They also detoxify many poisons that may enter the body. (Many of these oxidation reactions release hydrogen peroxide, H2O2, which would be damaging to cells; however, when these reactions are confined to peroxisomes, enzymes safely break down the H2O2 into oxygen and water.) For example, peroxisomes in liver cells detoxify alcohol. Glyoxysomes, which are specialized peroxisomes in plants, are responsible for converting stored fats into sugars. Plant cells contain many different types of peroxisomes that play a role in metabolism, pathogene defense, and stress response, to mention a few.'''


### Defining prompts

In [3]:
# NOTE: Lesson review question prompt
comb_rev_quest_gen_string = """You are a biology professor that prepares review/guiding \
questions to help your students learn a lesson. Write at least 5 free-response questions \
designed to help your students understand the lesson material. Overall, the questions you write \
should promote comprehensive learning and cover all the lesson material. Please try \
to avoid writing questions that overlap in content. The lesson material is provided \
below delimited by triple backticks.\n Write the questions in the following format:\n\
Question 1: <Question 1 text> \n\
... \n\
Question N: <Question N text> \n\n\
Lesson Material:```{content}```"""
comb_rev_quest_template = ChatPromptTemplate.from_template(comb_rev_quest_gen_string)
print(comb_rev_quest_gen_string)

You are a biology professor that prepares review/guiding questions to help your students learn a lesson. Write at least 5 free-response questions designed to help your students understand the lesson material. Overall, the questions you write should promote comprehensive learning and cover all the lesson material. Please try to avoid writing questions that overlap in content. The lesson material is provided below delimited by triple backticks.
 Write the questions in the following format:
Question 1: <Question 1 text> 
... 
Question N: <Question N text> 

Lesson Material:```{content}```


In [4]:
prompt = comb_rev_quest_template.format_messages(content=LESSON_TEXT)
rev_quest = chat(prompt).content
print(rev_quest)

Question 1: What does the phrase "form follows function" mean in the context of cell biology? Provide an example to illustrate your answer.
Question 2: Describe the structure and function of ribosomes. How do they contribute to protein synthesis?
Question 3: Explain why mitochondria are often referred to as the "powerhouses" of cells. What is their role in energy production and how does this relate to their structure?
Question 4: What are peroxisomes and what is their function within a cell? How do they contribute to the detoxification process?
Question 5: Discuss the relationship between the number of organelles in a cell and the cell's function. Use the examples of muscle cells and liver cells in your explanation.


In [5]:
REV_QUESTS = [s.split(": ")[1] for s in rev_quest.split("\n") if len(s) > 5]
REV_QUESTS

['What does the phrase "form follows function" mean in the context of cell biology? Provide an example to illustrate your answer.',
 'Describe the structure and function of ribosomes. How do they contribute to protein synthesis?',
 'Explain why mitochondria are often referred to as the "powerhouses" of cells. What is their role in energy production and how does this relate to their structure?',
 'What are peroxisomes and what is their function within a cell? How do they contribute to the detoxification process?',
 "Discuss the relationship between the number of organelles in a cell and the cell's function. Use the examples of muscle cells and liver cells in your explanation."]

In [6]:
COMB_STRING = ""
for i, q in enumerate(REV_QUESTS):
    COMB_STRING += "Question " + str(i + 1) + ": " + q + "\n"
print(COMB_STRING)

Question 1: What does the phrase "form follows function" mean in the context of cell biology? Provide an example to illustrate your answer.
Question 2: Describe the structure and function of ribosomes. How do they contribute to protein synthesis?
Question 3: Explain why mitochondria are often referred to as the "powerhouses" of cells. What is their role in energy production and how does this relate to their structure?
Question 4: What are peroxisomes and what is their function within a cell? How do they contribute to the detoxification process?
Question 5: Discuss the relationship between the number of organelles in a cell and the cell's function. Use the examples of muscle cells and liver cells in your explanation.



In [7]:
# NOTE: solution generation
rev_quest_gen_string = """You are a biology professor that prepares solutions \
for a range of review/guiding questions designed to help your students learn a lesson. \
The questions and lesson text are provided below delimited by triple backticks. \
The solutions should be focused and target only the most important information from the lesson material. Write the solutions in the following format:\n\
Solution 1: <Question 1 solution text> \n\
... \n\
Solution N: <Question N solution text> \n\

Question List: ```{question}```\n\n\
Lesson Material: ```{content}```
"""
rev_answer_template = ChatPromptTemplate.from_template(rev_quest_gen_string)
print(rev_quest_gen_string)

You are a biology professor that prepares solutions for a range of review/guiding questions designed to help your students learn a lesson. The questions and lesson text are provided below delimited by triple backticks. The solutions should be focused and target only the most important information from the lesson material. Write the solutions in the following format:
Solution 1: <Question 1 solution text> 
... 
Solution N: <Question N solution text> 

Question List: ```{question}```

Lesson Material: ```{content}```



In [8]:
prompt = rev_answer_template.format_messages(question=COMB_STRING, content=LESSON_TEXT)
rev_sol = chat(prompt).content
print(rev_sol)

Solution 1: The phrase "form follows function" in cell biology means that the structure of cell components and organelles is directly related to their function. For example, ribosomes, which are responsible for protein synthesis, are found in high numbers in cells that synthesize large amounts of protein, such as those in the pancreas that produce digestive enzymes. Here, the form (high number of ribosomes) follows the function (protein synthesis).

Solution 2: Ribosomes are cellular structures responsible for protein synthesis. They are large protein and RNA complexes, each consisting of two subunits, large and small. Ribosomes may group together into clusters or float freely in the cytoplasm. They may also be attached to the plasma membrane's cytoplasmic side or the endoplasmic reticulum's cytoplasmic side and the nuclear envelope's outer membrane. Ribosomes receive their “orders” for protein synthesis from the nucleus where the DNA transcribes into messenger RNA (mRNA). The mRNA tra

In [9]:
SOLUTIONS = [s.split(": ")[1] for s in rev_sol.split("\n") if len(s) > 5]
SOLUTIONS

['The phrase "form follows function" in cell biology means that the structure of cell components and organelles is directly related to their function. For example, ribosomes, which are responsible for protein synthesis, are found in high numbers in cells that synthesize large amounts of protein, such as those in the pancreas that produce digestive enzymes. Here, the form (high number of ribosomes) follows the function (protein synthesis).',
 "Ribosomes are cellular structures responsible for protein synthesis. They are large protein and RNA complexes, each consisting of two subunits, large and small. Ribosomes may group together into clusters or float freely in the cytoplasm. They may also be attached to the plasma membrane's cytoplasmic side or the endoplasmic reticulum's cytoplasmic side and the nuclear envelope's outer membrane. Ribosomes receive their “orders” for protein synthesis from the nucleus where the DNA transcribes into messenger RNA (mRNA). The mRNA travels to the ribosom

In [10]:
# What happes if we create all expectations at once?
QS_STRING = ""
for i, q in enumerate(REV_QUESTS):
    QS_STRING += "Question " + str(i + 1) + ": " + q + "\n"
    QS_STRING += "Solution " + str(i + 1) + ": " + SOLUTIONS[i] + "\n"
print(QS_STRING)

Question 1: What does the phrase "form follows function" mean in the context of cell biology? Provide an example to illustrate your answer.
Solution 1: The phrase "form follows function" in cell biology means that the structure of cell components and organelles is directly related to their function. For example, ribosomes, which are responsible for protein synthesis, are found in high numbers in cells that synthesize large amounts of protein, such as those in the pancreas that produce digestive enzymes. Here, the form (high number of ribosomes) follows the function (protein synthesis).
Question 2: Describe the structure and function of ribosomes. How do they contribute to protein synthesis?
Solution 2: Ribosomes are cellular structures responsible for protein synthesis. They are large protein and RNA complexes, each consisting of two subunits, large and small. Ribosomes may group together into clusters or float freely in the cytoplasm. They may also be attached to the plasma membrane's

In [11]:
# NOTE: expectation generation
exp_quest_gen_string = """You are a biology professor that creates lists summarizing \
the key information elements contained in the solutions to review/guiding questions \
designed to help your students learn a lesson. \
The questions and solutions are provided below delimited by triple backticks. \
You want to keep the lists brief and focused. \
Write the lists in the following format:\n\
List 1: <Question 1 element 1; ...; Question 1 element m1>\n\
...\n\
List N: <Question N element 1; ...; Question N element mN>\n\
\n\
Questions and Solutions: ```{questions}```
"""
exp_answer_template = ChatPromptTemplate.from_template(exp_quest_gen_string)

In [12]:
prompt = exp_answer_template.format_messages(questions=QS_STRING)
rev_sol = chat(prompt).content
print(rev_sol)


List 1: "Form follows function" in cell biology; Structure of cell components and organelles related to their function; Example of ribosomes in cells that synthesize large amounts of protein
List 2: Structure and function of ribosomes; Role in protein synthesis; Ribosomes as large protein and RNA complexes; Ribosomes' location in the cell; Process of protein synthesis from DNA transcription to mRNA translation
List 3: Mitochondria as "powerhouses" of cells; Role in energy production; Structure of mitochondria; Process of cellular respiration; Example of muscle cells with high concentration of mitochondria
List 4: Structure and function of peroxisomes; Role in oxidation reactions and detoxification; Process of breaking down hydrogen peroxide; Example of peroxisomes in liver cells detoxifying alcohol
List 5: Relationship between number of organelles and cell's function; Example of muscle cells with high concentration of mitochondria; Example of liver cells with high number of peroxisomes

In [13]:
# assemble expectation script
formated = [s.split(": ")[1] for s in rev_sol.split("\n") if len(s) > 5]

for i, point in enumerate(formated):
    print("Topic " + str(i + 1) + ": " + REV_QUESTS[i].replace('"', "''"))
    for j, fact in enumerate(point.split(";")):
        fact = fact.replace('"', "''")
        fact = fact[min(j, 1):]
        print("Fact " + str(i + 1) + "." + str(j + 1) + ": " + fact + ".")
    print("")

Topic 1: What does the phrase ''form follows function'' mean in the context of cell biology? Provide an example to illustrate your answer.
Fact 1.1: ''Form follows function'' in cell biology.
Fact 1.2: Structure of cell components and organelles related to their function.
Fact 1.3: Example of ribosomes in cells that synthesize large amounts of protein.

Topic 2: Describe the structure and function of ribosomes. How do they contribute to protein synthesis?
Fact 2.1: Structure and function of ribosomes.
Fact 2.2: Role in protein synthesis.
Fact 2.3: Ribosomes as large protein and RNA complexes.
Fact 2.4: Ribosomes' location in the cell.
Fact 2.5: Process of protein synthesis from DNA transcription to mRNA translation.

Topic 3: Explain why mitochondria are often referred to as the ''powerhouses'' of cells. What is their role in energy production and how does this relate to their structure?
Fact 3.1: Mitochondria as ''powerhouses'' of cells.
Fact 3.2: Role in energy production.
Fact 3.3: 

In [14]:
# assemble LLM_QA
QATopicsLLM = "[\n"
for i, _, in enumerate(REV_QUESTS):
    QATopicsLLM += "  {\n"
    QATopicsLLM += '    question: "' + REV_QUESTS[i].replace('"', "''") + '",\n'
    QATopicsLLM += '    solution: "' + SOLUTIONS[i].replace('"', "''") + '",\n'
    QATopicsLLM += "  },\n"
QATopicsLLM += "];\n"

print(QATopicsLLM)

[
  {
    question: "What does the phrase ''form follows function'' mean in the context of cell biology? Provide an example to illustrate your answer.",
    solution: "The phrase ''form follows function'' in cell biology means that the structure of cell components and organelles is directly related to their function. For example, ribosomes, which are responsible for protein synthesis, are found in high numbers in cells that synthesize large amounts of protein, such as those in the pancreas that produce digestive enzymes. Here, the form (high number of ribosomes) follows the function (protein synthesis).",
  },
  {
    question: "Describe the structure and function of ribosomes. How do they contribute to protein synthesis?",
    solution: "Ribosomes are cellular structures responsible for protein synthesis. They are large protein and RNA complexes, each consisting of two subunits, large and small. Ribosomes may group together into clusters or float freely in the cytoplasm. They may also

## EVERYTHING BELOW IS DEPRECATED

### GPT3.5 generated

In [ ]:
"""
Q1

Question: How does the principle of "form follows function" apply to eukaryotic cells and their organelles?

Question: What is the main difference between eukaryotic and prokaryotic cells?
Option A: Eukaryotic cells have a membrane-bound nucleus and numerous membrane-bound organelles, while prokaryotic cells do not.
Option B: Eukaryotic cells have several rod-shaped chromosomes, while prokaryotic cells have only one circular chromosome.
Option C: Eukaryotic cells have a cell wall, while prokaryotic cells do not.
Option D: Eukaryotic cells have a flagellum for movement, while prokaryotic cells do not.
Correct Option: A

Q2

What is the function of the plasma membrane in eukaryotic cells and how does it control the passage of molecules in and out of the cell?

Question: What is the function of the plasma membrane in eukaryotic cells?
Option A: To absorb nutrients from digested food
Option B: To control the passage of organic molecules, ions, water, and oxygen into and out of the cell
Option C: To produce energy for the cell
Option D: To protect the cell from external threats
Correct Option: B


Q3

What are the different types of organic molecules and elements found in the cytoplasm, and what metabolic reactions occur in this region of the cell?

Question: Which of the following is NOT found in the cytoplasm of a cell?
Option A: Glucose
Option B: Proteins
Option C: Calcium ions
Option D: DNA
Correct Option: D


Q4

What is the function of the nucleus in a cell and what does it house?

Question: Which organelle houses the cell's DNA and directs the synthesis of ribosomes and proteins?
Option A: Mitochondria
Option B: Golgi Apparatus
Option C: Nucleus
Option D: Endoplasmic Reticulum
Correct Option: C


Q5

How do chromosomes differ between prokaryotes and eukaryotes, and what is chromatin?

Question: What is chromatin?
Option A: A circular structure made up of DNA in prokaryotes.
Option B: A linear structure made up of DNA in eukaryotes.
Option C: The material that makes up chromosomes both when condensed and decondensed.
Option D: A protein that attaches to chromosomes during cell division.
Correct Option: C


Q6

What is the function of ribosomes in protein synthesis and how do they receive their "orders" for protein synthesis?
    
Question: What is the function of ribosomes in a cell?
Option A: To store genetic information
Option B: To produce lipids
Option C: To synthesize proteins
Option D: To break down waste products


Q7

Why are mitochondria often referred to as "powerhouses" or "energy factories" of both plant and animal cells?

Question: What is the main function of mitochondria in both plant and animal cells?
Option A: To produce glucose
Option B: To produce carbon dioxide
Option C: To store long-term energy
Option D: To produce ATP
Correct Option: D


Q8

What are some of the functions of peroxisomes in cells?

Question: What is the function of peroxisomes in liver cells?
Option A: Break down sugars
Option B: Convert stored fats into sugars
Option C: Detoxify alcohol
Option D: Carry out photosynthesis
Correct Option: C


Q9

What is the subtle distinction between vesicles and vacuoles, and how do their functions differ?

Question: What is the main difference between vesicles and vacuoles?
Option A: Vesicles are larger than vacuoles.
Option B: Vesicle membranes can fuse with other membrane systems within the cell, while vacuole membranes do not.
Option C: Vacuoles are used for storage, while vesicles are used for transport.
Option D: Vesicles support cell expansion, while vacuoles break down macromolecules.
Correct Option: B
"""

In [ ]:
content = """
Peroxisomes are small, round organelles enclosed by single membranes. They carry out oxidation reactions that break down fatty acids and amino acids. They also detoxify many poisons that may enter the body. (Many of these oxidation reactions release hydrogen peroxide, H2O2, which would be damaging to cells; however, when these reactions are confined to peroxisomes, enzymes safely break down the H2O2 into oxygen and water.) For example, peroxisomes in liver cells detoxify alcohol. Glyoxysomes, which are specialized peroxisomes in plants, are responsible for converting stored fats into sugars. Plant cells contain many different types of peroxisomes that play a role in metabolism, pathogene defense, and stress response, to mention a few.`;
"""
print("Summary:\n----------------------------------------")
prompt = sum_content_template.format_messages(content=content)
summary_list = chat(prompt).content
print(summary_list)

#### Summary prompt:

You are a biology college professor that teaches first year undergraduate students. Please help write a one sentence question that promotes a discussion about the knowledge that the questions below inquire about:

In [ ]:
MAN_QS = [
    "Explain the principle of form follows function. Provide at least 2 examples of how this principle applies with respect to the structure of eukaryotic cells.",
    "What is the function of a membrane? Which structures/organelles of a eukaryotic cell are bounded by one or more membranes? Why is it important that these organelles are bounded in this way?",
    "What is the consistency of the cytoplasm? Why? What is found within the cytoplasm of a eukaryotic cell? What happens here?",
    "What are ribosomes composed of? Where in a cell are ribosomal subunits made and assembled? Where are they used? What are they used for? What provides them the instructions for their function?",
    "How does breathing relate to the process of cellular respiration? What does cellular respiration use and produce? Where does it occur in eukaryotic cells?"
]

MAN_ANSW = [
    "The principle of form follows function means that the function of a particular structure is specifically designed to help the structure carry out its particular function. There are many examples where a type of eukaryotic cell is enriched in a particular structure or organelle to optimize its ability to perform a particular function. These include (but only 2 need to be provided): A)	Cells lining the small intestine have microvilli (increased amounts of plasma membrane found in folds), increasing their area for nutrient absorption. When these microvilli are damaged (as occurs during celiac disease), nutrient absorption is disrupted. B) Cells that need to make a lot of proteins, like pancreatic cells that make a high quantity of digestive enzymes, contain may ribosomes, the structure responsible for protein synthesis. C) cells have a very high concentration of mitochondria in order to make high amounts of ATP (the cell’s short-term stored energy) to keep your body moving.",
    "A membrane separates the internal contents of a cell or organelle from its surroundings, controlling the passage of materials from one side of the membrane to the other. A phospholipid bilayer with embedded proteins composes the plasma membrane, and several organelles, including the nucleus (surrounded by a double membrane), mitochondria (each surrounded by a double membrane), and peroxisomes (each surrounded by a single membrane) are also bound by membranes. It is important that the organelles are bounded in this way so that the materials they contain, and their functions are compartmentalized within the larger eukaryotic cell.",
    "While the cytoplasm consists of 70-80% water, it has a gel-like, semi-solid consistency due to the proteins found within it. Beyond the cytosol, in which the organelles are suspended, the cytoplasm also contains the cytoskeleton and various chemicals, including glucose, other sugars, amino acids, nucleic acids, fatty acids, glycerol derivatives, and many dissolved ions and elements. The cytoplasm is where many metabolic reactions including protein synthesis.",
    "Each ribosome is composed of two subunits, a large and a small, each of which contains both proteins and RNA. Ribosomal RNA is made in the nucleus, and the ribosomal RNA and ribosomal proteins assemble into ribosomal subunits in the nucleolus within the nucleus. Then, the ribosomal subunits leave the nucleus through the nuclear envelope’s pores to enter the cytoplasm. Once outside of the nucleus, ribosomes may group as clusters or remain individual ribosomes that freely float in the cytoplasm. Additionally, ribosomes may also attach to the cytoplasmic side of the endoplasmic reticulum or the nuclear envelope’s outer membrane. They are used for linking amino acids together in a particular order to make proteins, using the information within mRNA (which is made using DNA in the nucleus) to direct the identity and order of amino acids in the protein.",
    "In breathing, the body takes in oxygen during inhalation and releases carbon dioxide during exhalation. This is related to cellular respiration, which uses oxygen to make adenosine triphosphate (ATP), a form of short-term energy storage, and produces carbon dioxide as a byproduct. Thus, breathing provides the oxygen that the body’s cells need for cellular respiration and removes the carbon dioxide they produce as waste. In eukaryotic cells, cellular respiration occurs in the mitochondria."
]

In [ ]:
for i, _ in enumerate(MAN_QS):

    print("\nExpectations for answer\n----------------------------------------")
    print(i)
    prompt = exp_answer_template.format_messages(question=MAN_QS[i], answer=MAN_ANSW[i])
    rev_expectations = chat(prompt).content
    print(rev_expectations)


#### Old single question generation

In [ ]:
# NOTE: review question generation (for student bot)
rev_quest_gen_string = """You are a biology professor that prepares a review \
question to help your students understand a specific piece of content. \
The content is provided below delimited by triple backticks. You want to avoid
composite questions. Write the question in the following format:\n\
Question: [question text]\n\
\n\
Content: ```{content}```
"""
rev_quest_template = ChatPromptTemplate.from_template(rev_quest_gen_string)

In [ ]:
# NOTE: multiple-choice question generation
mult_quest_gen_string = """You are a biology professor that prepares a \
multiple-choice question to test your students' understanding of a specific \
piece of content. The question has four answer options and a single correct \
answer. The content is provided below delimited by triple backticks. Write \
the question in the following format:\n\
Question: [question text]\n\
Option A: [option text]\n\
Option B: [option text]\n\
Option C: [option text]\n\
Option D: [option text]\n\
Correct Option: [option letter]\n\
\n\
Content: ```{content}```
"""
mult_quest_template = ChatPromptTemplate.from_template(mult_quest_gen_string)

In [ ]:
# NOTE: solution generation
rev_quest_gen_string = """You are a biology professor that prepares a model solution \
to help your students understand a review questions that targets a specific piece of content. \
The review question and content are provided below delimited by triple backticks. \
The model solution should be brief (<40 words) but still capture the important \
information. Write the answer in the following format:\n\
Answer: [Answer Text]\n\
\n\
Review Question: ```{question}```
\n\
Content: ```{content}```
"""
rev_answer_template = ChatPromptTemplate.from_template(rev_quest_gen_string)

In [ ]:
# NOTE: Lesson review question prompt
comb_rev_quest_gen_string = """You are a biology professor that prepares review/guiding \
questions to help your students learn a lesson. Write at least 5 free-response questions \
designed to help your students understand the lesson material. Overall, the questions you write \
should promote comprehensive learning and cover all the lesson material. Please try \
to avoid writing questions that overlap in content. The lesson material is provided \
below delimited by triple backticks.\n\
Lesson Material:```{content}```"""
comb_rev_quest_template = ChatPromptTemplate.from_template(comb_rev_quest_gen_string)

In [ ]:
# one at a time expectation generation

# NOTE: expectation generation
exp_quest_gen_string = """You are a biology professor that creates a list to summarize \
the key information elements contained in the solution to a review/guiding question \
designed to help your students learn a lesson. \
The question and solution are provided below delimited by triple backticks. \
You want to keep the list and descriptions brief and focused. \
Write the list in the following format:\n\
Element 1: <Element 1 description>\n\
...\n\
Element N: <Element N description>\n\
\n\
Question: ```{question}```
\n\
Solution: ```{solution}```
"""
exp_answer_template = ChatPromptTemplate.from_template(exp_quest_gen_string)

print(exp_quest_gen_string)

for i, q in enumerate(REV_QUESTS):
    print("Question", i, q)
    print(SOLUTIONS[i])
    prompt = exp_answer_template.format_messages(question=q, solution=SOLUTIONS[i])
    rev_sol = chat(prompt).content
    print(rev_sol)
    print("")
